## NBA Position Predictor Project

### Created 16 November 2021

## Creating a Model

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from get_stats import *

In [2]:
data = get_full_dataframe()

In [3]:
stats_df = data
[(i, list(set(stats_df[stats_df['Year'] == i]['POS']))) for i in range(2017, 2022)]  # test

[(2017, ['SG', 'PG', 'SF', 'PF', 'G', 'C']),
 (2018, ['C-F', 'G-F', 'G', 'F-C', 'F-G', 'F', 'C']),
 (2019, ['C-F', 'G-F', 'G', 'F-G', 'F-C', 'F', 'C']),
 (2020, ['C-F', 'G-F', 'G', 'F-C', 'F-G', 'F', 'C']),
 (2021, ['C-F', 'G-F', 'G', 'F-C', 'F-G', 'F', 'C'])]

In [4]:
stats_df = stats_df[stats_df['Year'] != 2017].copy()
stats_df

,RANK,NAME,TEAM,POS,AGE,GP,MPG,MIN%,USG%,TO%,...,TRB%,APG,AST%,SPG,BPG,TOPG,VI,ORTG,DRTG,Year
209,NaN,Marc Gasol,Tor,C,34.37,24,30.6,63.8,13.6,9.5,...,11.5,3.0,14.7,0.88,1.08,0.88,7.4,118.4,97.2,2018
210,NaN,Danny Green,Tor,G-F,31.98,24,28.5,59.4,12.6,13.6,...,7.0,1.1,5.4,1.29,0.46,1.08,4.2,97.4,101.6,2018
211,NaN,Serge Ibaka,Tor,F-C,29.74,24,20.8,43.4,21.4,10.6,...,15.8,0.9,7.4,0.46,0.96,1.04,7.1,105.5,91.9,2018
212,NaN,Kawhi Leonard,Tor,F,27.96,24,39.1,81.6,32.0,11.1,...,12.8,3.9,19.3,1.67,0.71,3.08,10.5,118.9,102.7,2018
213,NaN,Kyle Lowry,Tor,G,33.22,24,37.5,78.2,18.4,14.5,...,7.2,6.6,27.5,1.29,0.29,2.21,8.4,116.8,100.1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,NaN,Omer Yurtseven,Mia,C,23.74,49,12.8,26.6,20.2,13.3,...,23.8,0.9,11.9,0.31,0.39,0.78,9.5,114.6,92.9,2021
1568,NaN,Cody Zeller,Por,F-C,29.44,27,13.1,27.4,16.0,14.6,...,19.2,0.8,9.4,0.30,0.22,0.70,8.2,128.8,100.0,2021
1569,NaN,Ivica Zubac,Lac,C,25.00,65,24.7,51.4,16.6,16.9,...,17.9,1.5,9.2,0.51,1.05,1.58,8.3,126.7,99.3,2021
1570,NaN,Tyrone Wallace,Nor,G,27.77,2,17.9,37.3,13.5,0.0,...,6.2,0.0,0.0,0.00,0.50,0.00,0.0,90.3,106.7,2021


In [5]:
# replace positions to get two different player classifications:
pos_encode = {"G":0 , "F":1, "C":2, "GF":3, "FC":4}
## pos_encode = {"PG":0 , "SF":1, "C":2, "SG":3, "PF":4, "G":5 , "F":6}
pos_decode = {0:"G", 1:"F", 2:"C", 3:"GF", 4:"FC"}
## pos_decode = {0:"PG", 1:"SF", 2:"C", 3:"SG", 4:"PF"}

# - POS3 = {"G", "F", "C"}  # - POS5 = {"G", "GF", "F", "FC", "C"}
stats_df.loc[:, "POS3"] = stats_df["POS"].replace("G-F", "G").replace("F-G", "F").replace("C-F", "C").replace("F-C", "F")
## stats_df.loc[:, "POS3"] = stats_df["POS3"].replace("PG", "G").replace("SG", "G").replace("SF", "F").replace("PF", "F")
stats_df.loc[:, "POS5"] = stats_df["POS"].replace("G-F", "GF").replace("F-G", "GF").replace("C-F", "FC").replace("F-C", "FC")
## stats_df.loc[:, "POS5"] = stats_df["POS"].replace("G-F", "SG").replace("F-G", "SG").replace("C-F", "PF").replace("F-C", "PF")
## stats_df.loc[:, "POS5"] = stats_df["POS5"].replace("G", "PG").replace("F", "SF")
stats_df

,RANK,NAME,TEAM,POS,AGE,GP,MPG,MIN%,USG%,TO%,...,AST%,SPG,BPG,TOPG,VI,ORTG,DRTG,Year,POS3,POS5
209,NaN,Marc Gasol,Tor,C,34.37,24,30.6,63.8,13.6,9.5,...,14.7,0.88,1.08,0.88,7.4,118.4,97.2,2018,C,C
210,NaN,Danny Green,Tor,G-F,31.98,24,28.5,59.4,12.6,13.6,...,5.4,1.29,0.46,1.08,4.2,97.4,101.6,2018,G,GF
211,NaN,Serge Ibaka,Tor,F-C,29.74,24,20.8,43.4,21.4,10.6,...,7.4,0.46,0.96,1.04,7.1,105.5,91.9,2018,F,FC
212,NaN,Kawhi Leonard,Tor,F,27.96,24,39.1,81.6,32.0,11.1,...,19.3,1.67,0.71,3.08,10.5,118.9,102.7,2018,F,F
213,NaN,Kyle Lowry,Tor,G,33.22,24,37.5,78.2,18.4,14.5,...,27.5,1.29,0.29,2.21,8.4,116.8,100.1,2018,G,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,NaN,Omer Yurtseven,Mia,C,23.74,49,12.8,26.6,20.2,13.3,...,11.9,0.31,0.39,0.78,9.5,114.6,92.9,2021,C,C
1568,NaN,Cody Zeller,Por,F-C,29.44,27,13.1,27.4,16.0,14.6,...,9.4,0.30,0.22,0.70,8.2,128.8,100.0,2021,F,FC
1569,NaN,Ivica Zubac,Lac,C,25.00,65,24.7,51.4,16.6,16.9,...,9.2,0.51,1.05,1.58,8.3,126.7,99.3,2021,C,C
1570,NaN,Tyrone Wallace,Nor,G,27.77,2,17.9,37.3,13.5,0.0,...,0.0,0.00,0.50,0.00,0.0,90.3,106.7,2021,G,G


In [6]:
print(stats_df.groupby('POS3')['POS3'].count())
print(stats_df.groupby('POS5')['POS5'].count())

POS3
C    143
F    479
G    582
Name: POS3, dtype: int64
POS5
C      97
F     345
FC    130
G     473
GF    159
Name: POS5, dtype: int64


In [7]:
stats_df.describe()

,RANK,AGE,GP,MPG,MIN%,USG%,TO%,FTA,FT%,2PA,...,TRB%,APG,AST%,SPG,BPG,TOPG,VI,ORTG,DRTG,Year
count,0.0,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,...,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000
mean,NaN,27.263098,22.132060,20.699086,43.116445,18.549751,11.772841,46.855482,0.682197,112.055648,...,10.115116,1.966694,13.109718,0.618978,0.392542,1.097741,6.500166,108.497259,104.205565,2020.065615
std,NaN,4.185757,20.769206,10.311067,21.483654,6.227288,6.641676,73.209074,0.273329,159.839210,...,5.181873,1.984930,10.097444,0.467155,0.421823,0.936020,2.833126,21.422068,6.378397,1.136494
min,NaN,19.230000,1.000000,4.000000,8.300000,4.300000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.200000,2018.000000
25%,NaN,23.922500,5.000000,11.675000,24.350000,14.200000,8.100000,5.000000,0.611750,13.000000,...,6.400000,0.600000,6.500000,0.270000,0.110000,0.450000,5.100000,100.475000,100.800000,2019.000000
50%,NaN,26.965000,12.000000,20.050000,41.700000,17.600000,11.200000,18.000000,0.759500,44.000000,...,8.950000,1.300000,10.250000,0.535000,0.275000,0.830000,6.500000,110.300000,104.600000,2021.000000
75%,NaN,30.170000,40.000000,29.600000,61.700000,22.200000,14.700000,56.000000,0.850000,142.000000,...,13.000000,2.600000,17.800000,0.900000,0.530000,1.500000,8.100000,118.700000,108.500000,2021.000000
max,NaN,41.770000,70.000000,41.300000,86.100000,48.300000,50.000000,666.000000,1.000000,1179.000000,...,44.100000,11.800000,76.900000,2.800000,4.000000,5.170000,17.400000,280.100000,121.500000,2021.000000


In [8]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1204 entries, 209 to 1571
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RANK    0 non-null      float64
 1   NAME    1204 non-null   object 
 2   TEAM    1204 non-null   object 
 3   POS     1204 non-null   object 
 4   AGE     1204 non-null   float64
 5   GP      1204 non-null   int64  
 6   MPG     1204 non-null   float64
 7   MIN%    1204 non-null   float64
 8   USG%    1204 non-null   float64
 9   TO%     1204 non-null   float64
 10  FTA     1204 non-null   int64  
 11  FT%     1204 non-null   float64
 12  2PA     1204 non-null   int64  
 13  2P%     1204 non-null   float64
 14  3PA     1204 non-null   int64  
 15  3P%     1204 non-null   float64
 16  eFG%    1204 non-null   float64
 17  TS%     1204 non-null   float64
 18  PPG     1204 non-null   float64
 19  RPG     1204 non-null   float64
 20  TRB%    1204 non-null   float64
 21  APG     1204 non-null   float64
 22

### Support Vector Machine

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [10]:
#divide the data into the input 'X' and the labels 'y'
#selected_features = ['MIN%', 'USG%', 'TO%', 'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG',
#                     'RPG', 'TRB%','APG', 'AST%', 'SPG', 'BPG', 'TOPG', 'VI', 'ORTG', 'DRTG']
selected_features = [i for i in list(stats_df)[4:-3]]  # if i not in ['MPG', 'PPG']]
X        = stats_df[selected_features] #the observations
y        = stats_df['POS3'] #the label

In [11]:
def SupportVector(X, y, target_names):
    # split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=6, stratify = y)
    # Enter appropriate values to partition data, and ensure results are reproducible and obtain stratified samples


    # feature scaling
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std, X_test_std = sc.transform(X_train), sc.transform(X_test)    
    
    # initialize the classifier
    svm = SVC() #the default kernel is rbf
    svm.fit(X_train_std, y_train) #fit the data
    
    # predict the labels for the test set
    y_pred   = svm.predict(X_test_std)
    # print('The prediction is: {}'.format(y_pred))
    
    #evaluate the prediction accuracy
    print('The accuracy of the model is: {}'.format(svm.score(X_test_std, y_test)))

    #get the classification report
    print(classification_report(y_test, y_pred, target_names = target_names))

In [12]:
SupportVector(X, stats_df['POS3'], target_names = ['C', 'F', 'G'])
# SupportVector(X, stats_df['POS5'], target_names = ['C', 'F', 'FC', "G", "GF"])

The accuracy of the model is: 0.7265193370165746
              precision    recall  f1-score   support

           C       0.73      0.37      0.49        43
           F       0.65      0.67      0.66       144
           G       0.78      0.86      0.82       175

    accuracy                           0.73       362
   macro avg       0.72      0.63      0.66       362
weighted avg       0.72      0.73      0.72       362



### Random Forest

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [14]:
stats_df.loc[:, "POS3"] = stats_df["POS3"].apply(lambda x: pos_encode[x])
stats_df.loc[:, "POS5"] = stats_df["POS5"].apply(lambda x: pos_encode[x])

In [15]:
#divide the data into the input 'X' and the labels 'y'
#selected_features = ['MIN%', 'USG%', 'TO%', 'FTA', 'FT%', '2PA', '2P%', '3PA', '3P%', 'eFG%', 'TS%', 'PPG',
#                     'RPG', 'TRB%','APG', 'AST%', 'SPG', 'BPG', 'TOPG', 'VI', 'ORTG', 'DRTG']
selected_features = [i for i in list(stats_df)[4:-3]]  # if i not in ['MPG', 'PPG']]
X        = stats_df[selected_features] #the observations
y        = stats_df['POS3'] #the label

In [16]:
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=6, stratify = y)
# Enter appropriate values to partition data, and ensure results are reproducible and obtain stratified samples


# feature scaling
sc = StandardScaler()
sc.fit(X_train)
X_train_std, X_test_std = sc.transform(X_train), sc.transform(X_test)    


In [17]:
rf = RandomForestRegressor(n_estimators=10, random_state=7)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=7)

In [18]:
#predict the labels for the test set
y_pred   = rf.predict(X_test)
# print('The prediction is: {}'.format(y_pred))

# Evaluate the Predictions
mse = mean_squared_error(y_test, y_pred)
print('The mse of the model is: {}'.format(mse))

The mse of the model is: 0.2356077348066298


In [19]:
n = max(rf.feature_importances_)
param = selected_features[list(rf.feature_importances_).index(n)]
print(f"The most important parameter was '{param}'.")

The most important parameter was 'TRB%'.


In [20]:
# sorted([(n, selected_features[list(rf.feature_importances_).index(n)]) for n in rf.feature_importances_], key = lambda x: x[0])